[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/dbs/blob/main/01--postgresql.ipynb)

## Instalacja PostgreSQL

In [ ]:
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

In [ ]:
## 

### Tworzenie użytkownika, bazy oraz przyznawanie uprawnień

In [ ]:
!sudo -u postgres psql -c "create user dbuser WITH encrypted password 'dbpw'"

In [ ]:
!sudo -u postgres psql -c "DROP database IF EXISTS db"
!sudo -u postgres psql -c "create database db with OWNER dbuser"
!sudo -u postgres psql db -c "GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO dbuser"

In [ ]:
!pip install psycopg2

In [ ]:
import psycopg2

In [ ]:
connection = psycopg2.connect(user="dbuser",
                                  password="dbpw",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db")

In [ ]:
# Print PostgreSQL version
cursor = connection.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()

In [ ]:
print(record)

In [ ]:
create_table = '''
DROP TABLE IF EXISTS Employee;
CREATE TABLE IF NOT EXISTS Employee (
  id SERIAL PRIMARY KEY,
  name text NOT NULL,
  sallary money NOT NULL
)
'''

In [ ]:
def connect_db():
    connection = psycopg2.connect(user="dbuser",
                                  password="dbpw",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db")
    return connection

In [ ]:
try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(create_table)
    connection.commit()
except (Exception, psycopg2.DataError) as error:
    print ("Error while creating PostgreSQL table", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
insert_sql = '''
INSERT INTO Employee (name, sallary) VALUES (%s,%s)
''';

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(insert_sql, ("jakub", 2500.00))
    connection.commit()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
select = '''
    select * from Employee;
'''

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(select)
    records = cursor.fetchall()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
print(records)

In [ ]:
!mkdir -p var/
!wget https://s3.eu-central-1.amazonaws.com/jkan.pl/uek-krakow/finanse-obliczeniowe/retail_sales.gz -O var/retail_sales.gz
!wget https://s3.eu-central-1.amazonaws.com/jkan.pl/uek-krakow/finanse-obliczeniowe/time+series+queries.sql

In [ ]:
!gunzip < var/retail_sales.gz | sudo -u postgres psql db

In [ ]:
!sudo -u postgres psql db -c "select * from retail_sales" &